# Introduction

We propose to achieve movie genre classification based only on movie poster images. A deep neural network is constructed to jointly describe visual appearance and object information, and classify a given movie poster image into genres. Because a movie may belong to multiple genres, this is a multi-label image classification problem. To facilitate related studies, we collect a large-scale movie poster dataset, associated with various metadata. Based on this dataset, we fine-tune a pre-trained convolutional neural network to extract visual representation and adopt a state-of-the-art framework to detect objects in posters. Two types of information are then integrated by the proposed neural network. In the evaluation, we show that the proposed method yields encouraging performance, which is much better than previous works. 



In [0]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 38kB/s 
     |████████████████████████████████| 3.8MB 43.7MB/s 
     |████████████████████████████████| 450kB 50.7MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


# Multi-Label Classification

Each sample can belong to more than one class. The CNN will have as well 
C
 output neurons. The target vector 
t
 can have more than a positive class, so it will be a vector of 0s and 1s with 
C
 dimensionality.
This task is treated as 
C
 different binary 
(
C
′
=
2
,
t
′
=
0
 or 
t
′
=
1
)
 and independent classification problems, where each output neuron decides if a sample belongs to a class or not.
 
(![alt text](https://gombru.github.io/assets/cross_entropy_loss/multiclass_multilabel.png))

![alt text](https://)# New Section

In [0]:
import pandas as pd
import json
import glob  
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPool2D,Dropout


In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [0]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [6]:
print(tf.__version__)

1.15.0


In [8]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from zipfile import ZipFile
# Create a ZipFile Object and load Movie_poster_dataset.zip in it
with ZipFile('/content/drive/My Drive/Movie_Poster_Dataset.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/content/Multiclassification_Genre/images')

In [0]:
from zipfile import ZipFile
# Create a ZipFile Object and load Movie_poster_Metadata in it
with ZipFile('/content/drive/My Drive/Movie_Poster_Metadata.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/content/Multiclassification_Genre/MoviesList')

In [0]:
#The JSON files that are present are not well formated, converting it to well formated file.

ff=open("/content/Multiclassification_Genre/MoviesList/groundtruth/1980.txt","r")
Movies=ff.read()
comma=Movies.replace("}","},")
newc=comma[:-2]
sqaureAdd="["+newc+"]"
aaa=sqaureAdd.replace("\n","")
jj=aaa.replace("ObjectId(","").replace("\")","\"")
fin=json.loads(jj)
df = pd.DataFrame.from_dict(fin, orient='columns')

In [0]:
#Framing the layout
cols=df.columns
dfFin=pd.DataFrame(columns=cols)
cols=df.columns
dfFin=pd.DataFrame(columns=cols)
dfFin
dfFin2=dfFin

In [0]:
#Importing all the text files, since there are 2 types of text files available ,
#using a exception block to handle both the file formats

all_files = glob.glob("/content/Multiclassification_Genre/MoviesLis/*.txt")

print((all_files))
count=0
for files in all_files:
  try:
    ff=open(files,"r",encoding='utf-16')
    stringg=ff.read()
    comma=Movies.replace("}","},")
    newc=comma[:-2]
    sqaureAdd="["+newc+"]"
    aaa=sqaureAdd.replace("\n","")
    jj=aaa.replace("ObjectId(","").replace("\")","\"")
    fin=json.loads(jj)
    df = pd.DataFrame.from_dict(fin, orient='columns')
    dfFin=pd.concat([dfFin,df])
  except:
    ff=open(files,"r",encoding='utf-8')
    stringg=ff.read()
    comma=Movies.replace("}","},")
    newc=comma[:-2]
    sqaureAdd="["+newc+"]"
    aaa=sqaureAdd.replace("\n","")
    jj=aaa.replace("ObjectId(","").replace("\")","\"")
    fin=json.loads(jj)
    df = pd.DataFrame.from_dict(fin, orient='columns')
    dfFin=pd.concat([dfFin,df])
    

In [0]:
#Merging both the dataframes
Movie_data=pd.concat([dfFin,dfFin2])
#Movie_data.head()
Movie_data


Data Exploration and Pre-processing

In [0]:
#Creating a subset of the the total data

Movie_Final=Movie_data[['imdbID','Genre',]]


In [0]:
Movie_Final.to_csv('Movie_genre.csv',encoding="ISO-8859-1")

In [0]:
Movie_genre=Movie_Final
Movie_genre['Genre']=[elem[0] for elem in Movie_genre['Genre']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
Moviegenrestats=Movie_genre.groupby('Genre').size().reset_index(name='counts')

In [0]:
Moviegenrestats=Moviegenrestats.sort_values('counts',ascending=False)

In [0]:
MovieFilter=list(Moviegenrestats)

In [0]:
#Getting the Movie Posters
image_paths = glob.glob("/content/Multiclassification_Genre/images/*.jpg")
image_ids = []
for path in image_paths:
    start = path.rfind("/")+1
    end = len(path)-4
    image_ids.append(path[start:end])

In [0]:
for fn in image_glob:
    try:
        img_dict[get_id(fn)] = scipy.misc.imread(fn)
    except:
        pass

In [0]:
#Showing the image ID's
def show_img(id):
    title = data[data["imdbId"] == int(id)]["Title"].values[0]
    genre = data[data["imdbId"] == int(id)]["Genre"].values[0]
    plt.imshow(img_dict[id])
    plt.title("{} \n {}".format(title, genre))

# Lets start Modelling

In [0]:
#Preprocessing function to scale the image…

def preprocess(img, size=(150, 101)):
    img = scipy.misc.imresize(img, size)
    img = img.astype(np.float32)
    img = (img / 127.5) - 1.
    return img

In [0]:
#Prepare the data
def prepare_data(data, img_dict, size=(150, 101)):
    print("Generation dataset...")
    dataset = []
    y = []
    ids = []
    label_dict = {"word2idx": {}, "idx2word": []}
    idx = 0
    genre_per_movie = Movie_final["Genre"].apply(lambda x: str(x).split("|"))
    for l in [g for d in genre_per_movie for g in d]:
        if l in label_dict["idx2word"]:
            pass
        else:
            label_dict["idx2word"].append(l)
            label_dict["word2idx"][l] = idx
            idx += 1
    n_classes = len(label_dict["idx2word"])
    print("identified {} classes".format(n_classes))
    n_samples = len(img_dict)
    print("got {} samples".format(n_samples))
    for k in img_dict:
        try:
            g = Movie_final[Movie_Final["imdbId"] == int(k)]["Genre"].values[0].split("|")
            img = preprocess(img_dict[k], size)
            if img.shape != (150, 101, 3):
                continue
            l = np.sum([np.eye(n_classes, dtype="uint8")[label_dict["word2idx"][s]] 
                                                        for s in g], axis=0)
            y.append(l)
            dataset.append(img)
            ids.append(k)
        except:
            pass
    print("DONE")
    return dataset, y, label_dict, ids

In [0]:
SIZE = (150, 101)
dataset, y, label_dict, ids =  prepare_data(data, img_dict, size=SIZE)

In [0]:
SIZE = (150, 101)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
                 input_shape=(SIZE[0], SIZE[1], 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(29, activation='sigmoid'))

we are using a sigmoid activation function with a multiclass output-layer. The sigmoid gives us independent propabilities for each class. So softmax is not used

In [20]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 99, 32)       896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 146, 97, 32)       9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 48, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 73, 48, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 71, 46, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 69, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 34, 22, 64)       

In [22]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.fit(np.array(dataset[: n]), np.array(y[: n]), batch_size=16, epochs=5,
          verbose=1, validation_split=0.1)

**References:**

1)https://www.analyticsvidhya.com/blog/2019/04/predicting-movie-genres-nlp-multi-label-classification/
2)https://www.depends-on-the-definition.com/classifying-genres-of-movies-by-looking-at-the-poster-a-neural-approach/
3)https://towardsdatascience.com/automated-movie-tagging-a-multiclass-classification-problem-721eb7fb70c2
4)https://www.youtube.com/watch?v=Vh26kOCra-Y

